# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [3]:
from computions import *

### get_describe_nominal function test

Basic situation

In [4]:
get_describe_nominal(test_frame.iloc[:,1], Y)

object_variable


,count,0.0,0.0%,2.0,2.0%,1.0,1.0%
object_variable,,,,,,,
6.0,122,4.0,3.278689,3.0,2.459016,115,94.262295
2.0,114,79.0,69.298246,0.0,0.000000,35,30.701754
1.0,111,95.0,85.585586,0.0,0.000000,16,14.414414
7.0,107,2.0,1.869159,16.0,14.953271,89,83.177570
4.0,97,33.0,34.020619,0.0,0.000000,64,65.979381
5.0,96,9.0,9.375000,1.0,1.041667,86,89.583333
8.0,96,0.0,0.000000,35.0,36.458333,61,63.541667
9.0,88,0.0,0.000000,36.0,40.909091,52,59.090909
3.0,87,48.0,55.172414,0.0,0.000000,39,44.827586


Data frame with emptys

In [5]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

object_variable
Is all obs. are in table -  False
object_variable
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{0.0: 0.28571225260441374, 2.0: 0.7669778526376146, 1.0: 0.562717258261934}
{'cetegory 0': 0.28571225260441374, 'category 2': 0.7669778526376146, 'category 1': 0.562717258261934}
{'cetegory 0': 0.2877590584345293, 'category 2': 0.7669333333333332, 'category 1': 0.5619941343994974}
{0.0: 0.28571225260441374, 1.0: 0.7142877473955862}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

object_variable
object_variable
{'cetegory 0': 0.9279445530275716, 'category 2': 0.9218302035550459, 'category 1': 0.827001223990208}
AUC bynary
object_variable
{0.0: 0.9279445530275716, 1.0: 0.9279445530275718}
AUC with emptys
{'cetegory 0': 0.07251165639323534, 'category 2': 0.9225824561403508, 'category 1': 0.6758633290699798}


Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

object_variable
nominal predictor without table
{0.0: {'AUC': 0.9279445530275716, 'rel_type': 1, 'GINI': 0.8558891060551432}, 2.0: {'AUC': 0.9218302035550459, 'rel_type': 1, 'GINI': 0.8436604071100917}, 1.0: {'AUC': 0.827001223990208, 'rel_type': 1, 'GINI': 0.6540024479804161}}
moninal predictor with table
{'cetegory 0': {'AUC': 0.9279445530275716, 'rel_type': 1, 'GINI': 0.8558891060551432}, 'category 2': {'AUC': 0.9218302035550459, 'rel_type': 1, 'GINI': 0.8436604071100917}, 'category 1': {'AUC': 0.827001223990208, 'rel_type': 1, 'GINI': 0.6540024479804161}}
numeric predictor
{0.0: {'AUC': 0.7142877473955862, 'rel_type': -1, 'GINI': 0.4285754947911724}, 2.0: {'AUC': 0.7669778526376146, 'rel_type': 1, 'GINI': 0.5339557052752293}, 1.0: {'AUC': 0.562717258261934, 'rel_type': 1, 'GINI': 0.12543451652386794}}


Decoding AUC info as DataFrame

In [9]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

0.0                          2.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.714288       -1  0.428575  0.766978        1  0.533956   

                       1.0                     
                       AUC rel_type      GINI  
numeric_variable  0.562717        1  0.125435

Getting indicators from computions uotput

In [10]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


cetegory 0                    category 2                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.712241       -1  0.424482   0.766933        1  0.533867   

                 category 1                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.561994        1  0.123988     20        0.02

## Writing to excel

Adding a new sheet

In [11]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')

for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [12]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

object_variable


Let's show final table

In [13]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

object_variable


0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.714288       -1  0.428575   
object_variable                                0.927945        1  0.855889   
column with soo000oo000oo000oo000oo long name  0.504279        1  0.008558   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.766978        1  0.533956   
object_variable                                 0.92183        1   0.84366   
column with soo000oo000oo000oo000oo long name  0.518537       -1  0.037074   

                                                    1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.562717        1  0.125435   
object_variable                                0.827001        1  0.654002   
column with soo000oo000oo000oo000oo long name  0.504761        1  0.009523   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [14]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

object_variable


Writing info about some predictor

In [15]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

object_variable


In [16]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()